In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 1. Carregar e preparar os dados
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ["mpg", "cylinders", "displacement", "horsepower", "weight", 
                "acceleration", "model year", "origin", "car name"]
df = pd.read_csv(url, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)



In [ ]:
#--------------------------- PRE PROCESSAMENTO ---------------------------#
df = df.drop('car name', axis=1)
df['origin'] = df['origin'].astype('category')
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
df = df.dropna()

print("Primeiras linhas do DataFrame após limpeza:")
print(df.head())

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#--------------------------- ANÁLISE EXPLORATÓRIA ---------------------------#
print("\nEstatísticas descritivas:")
print(df.describe().T)

# Matriz de correlação
plt.figure(figsize=(7, 5))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

# Pairplot para visualizar relações entre variáveis
sns.pairplot(df, vars=['mpg', 'weight', 'horsepower', 'acceleration'])
plt.figure(figsize=(5, 4))
plt.tight_layout()
plt.show()


In [ ]:
df.hist(bins=30, figsize=(10, 8))

In [ ]:
df['weight'] = df['weight'].apply(np.log10)
df['horsepower'] = df['horsepower'].apply(np.log10)
df["displacement"] = df["displacement"].apply(np.log10)
df.hist(bins=30, figsize=(10, 8))

In [ ]:
# -------------------------------Remoção de Outliers--------------------------------
# Utilizando Z-Score
# Z= (X−μ)/σ
# onde:
# X é o valor do ponto de dados,
# μ é a média do conjunto de dados,
# σ é o desvio padrão do conjunto de dados.

numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns.remove('mpg')  # Remover a variável alvo

def remove_outliers(df, columns, z_threshold):
    for col in columns:
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores < z_threshold]
    return df

print ("\nTamanho do DataFrame antes da remoção de outliers:")
print(df.shape)

# Quanto MAIOR o valor de z_threshold, MENOS outliers serão removidos
z_threshold = 3
df = remove_outliers(df, numeric_columns, z_threshold)

print("\nTamanho do DataFrame após remoção de outliers:")
print(df.shape)


In [ ]:
#--------------------------- FEATURE ENGENEERING ---------------------------#
print("Primeiras linhas do DataFrame antes de feature engineering:")
print(df.head())
# Normalização
scaler = StandardScaler()
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns.remove('mpg')
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# One-hot encoding para 'origin'
df = pd.get_dummies(df, columns=['origin'], prefix='origin')

# Criação de features polinomiais
X = df.drop('mpg', axis=1)
y = df['mpg']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
feature_names = poly.get_feature_names_out(X.columns)
X_poly = pd.DataFrame(X_poly, columns=feature_names)
print("\nPrimeiras linhas do DataFrame após feature engineering:")
print(X_poly.head())

#As features polinomiais ajudam a melhorar a capacidade do modelo de regressão de representar relações
#complexas nos dados, permitindo que ele capture comportamentos não lineares que uma abordagem linear 
#simples não conseguiria. Isso pode resultar em previsões mais precisas quando há uma relação não linear entre 
#as variáveis.

In [ ]:

#------------------------------- MODELAGEM -------------------------------#

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

print("""
Podem haver casos que X não possui inversa e portanto não é possivel calcular o theta. 
Por isso, o ridge adiciona um termo alpha*I onde I é a matriz identidade à matrix XtX
para garantir que ela seja inversível. 
O ideia é testar sem regularização e se não for possivel calcular o theta, usar a regularização.""")

# -------------------------------- MODELO USANDO NORMAL --------------------------------



def normal_equation_pseudoinverse(X, y):
    return np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)

# Adiciona a coluna de bias
X_train_with_bias = np.column_stack([np.ones(X_train.shape[0]), X_train])

# def normal_equation(X, y):
#     return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

# X_train_with_bias = np.column_stack([np.ones(X_train.shape[0]), X_train])
# theta_normal = normal_equation(X_train_with_bias, y_train)

# Calcula theta usando a pseudoinversa 
# Aqui temos uma matriz com os valores de theta para cada feature
theta_normal = normal_equation_pseudoinverse(X_train_with_bias, y_train)

# Equação Normal
X_test_ = np.column_stack([np.ones(X_test.shape[0]), X_test])
y_pred_normal = X_test_.dot(theta_normal)
mse_normal = mean_squared_error(y_test, y_pred_normal)
r2_normal = r2_score(y_test, y_pred_normal) 

# ------------------------------- MODELO USANDO SKLEARN -------------------------------

model_sklearn = LinearRegression()
model_sklearn.fit(X_train, y_train)

# theta_normal = np.concatenate([[model_sklearn.intercept_], model_sklearn.coef_])

y_pred_sklearn = model_sklearn.predict(X_test)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
r2_sklearn = r2_score(y_test, y_pred_sklearn)

# ------------------------------- MODELO USANDO RIDGE (L2) -------------------------------
from sklearn.linear_model import Ridge

# Define o valor de alpha para a regularização Ridge
alpha = 1e-3  # Este valor pode ser ajustado conforme necessário

# Criar o modelo de Ridge Regression
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X_train, y_train)

theta_normal = np.concatenate([[ridge_model.intercept_], ridge_model.coef_])

# Predição e avaliação com o modelo Ridge
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# ------------------------------- COMPARAÇÃO DE MODELOS -------------------------------

print("\nComparação de modelos:")

# Resultados da Equação Normal
print(f"Equação Normal: ")
print(f"MSE: {mse_normal:.4f}, R²: {r2_normal:.4f}")
# Printa a equação com os valores de theta
print(f"cyl*{theta_normal[1]:.4f} disp*{theta_normal[2]:.4f} hp*{theta_normal[3]:.4f} weight*{theta_normal[4]:.4f} ...")

# Resultados do Sklearn
print(f"\nSklearn: ")
print(f"MSE: {mse_sklearn:.4f}, R²: {r2_sklearn:.4f}")
print(f"cy*{model_sklearn.coef_[0]:.4f} disp*{model_sklearn.coef_[1]:.4f} hp*{model_sklearn.coef_[2]:.4f} weight*{model_sklearn.coef_[3]:.4f} ...")

# Resultados da Ridge Regression
print(f"\nRidge Regression: ")
print(f"MSE: {mse_ridge:.4f}, R²: {r2_ridge:.4f}")
print(f"cy*{ridge_model.coef_[0]:.4f} disp*{ridge_model.coef_[1]:.4f} hp*{ridge_model.coef_[2]:.4f} weight*{ridge_model.coef_[3]:.4f} ...")

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

feature_names = ['Intercept'] + list(X_train.columns)

# Criar DataFrame para os coeficientes
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': theta_normal})
coef_df = coef_df.sort_values('Coefficient', key=abs, ascending=False)

print("\nCoeficientes mais significativos:")
print(coef_df.head(10))

print("\nInterpretação dos coeficientes mais significativos:")
for _, row in coef_df.head(5).iterrows():
    print(f"{row['Feature']}: {row['Coefficient']:.4f}")
    print(f"  Uma mudança de uma unidade em {row['Feature']} está associada")
    print(f"  a uma mudança de {row['Coefficient']:.4f} em mpg, mantendo")
    print(f"  todas as outras variáveis constantes.\n")

In [ ]:
# Visualizations
# Actual vs. Predicted Values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_normal, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('MPG Real')
plt.ylabel('MPG Previsto')
plt.title('Valores Reais vs. Previstos (Equação Normal)')
plt.tight_layout()
plt.show()

# Residuals
residuals = y_test - y_pred_normal
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_normal, residuals)
plt.xlabel('Valores Previstos')
plt.ylabel('Resíduos')
plt.title('Gráfico de Resíduos (Equação Normal)')
plt.axhline(y=0, color='r', linestyle='--')
plt.tight_layout()
plt.show()

# Feature Importance
plt.figure(figsize=(12, 6))
coef_df.head(10).plot(x='Feature', y='Coefficient', kind='bar')
plt.title('Top 10 Coeficientes do Modelo (Equação Normal)')
plt.xlabel('Features')
plt.ylabel('Coeficiente')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()